In [ ]:
from heapq import merge

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy import feature as cfeature
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN

In [ ]:
equipment_data = pd.read_csv('../data/equipment.csv')
meldung_data = pd.read_csv('../data/Locker_Meldungen_Aufträge.csv')
anzahl_meldungen = pd.read_csv('../data/Anzahl_Meldungen.csv', delimiter=';')

In [ ]:
meldung_data.info()

In [ ]:
meldung_data.head()

In [ ]:
equipment_data.info()

In [ ]:
def extract_id(old: pd.Series) -> pd.Series:
    print(old.str.split(' ').str[1])
    new = old.str.split(' ').str[1]
    return new.astype(int)

In [ ]:
filter_i = meldung_data["Equipment"].str.contains("LÖVM")
meldung_data = meldung_data[~filter_i]

In [ ]:
anzahl_meldungen.Equipment = extract_id(anzahl_meldungen.Equipment)

In [ ]:
meldung_data.Equipment = extract_id(meldung_data.Equipment)

In [ ]:
meldung_data.info()

In [ ]:
anzahl_meldungen["Equipment"].astype(int)
anzahl_meldungen.info()

In [ ]:
#anzahl_meldungen = meldung_data.groupby("Equipment")["Meldung"].count().reset_index()
anzahl_meldungen.rename(columns={"Anzahl Sätze": "Anzahl Meldungen"}, inplace=True)

In [ ]:
merged_df = pd.merge(equipment_data, anzahl_meldungen, how='left', on='Equipment')

In [ ]:
merged_df.rename(columns={"Equipment": "Equipment ID"}, inplace=True)

In [ ]:
merged_df.info()

In [ ]:
columns_to_keep = ["Equipment ID", "Anzahl Meldungen", "lat", "lon", "Zyklus Wartung"]
merged_df = merged_df[columns_to_keep]

In [ ]:
# Replace Zyklus Wartung with Numbers
def zyklus_in_int(zyklus: str) -> int:
    if zyklus == "6M":
        zyklus = 6 * 30
    elif zyklus == "1J":
        zyklus = 365
    elif zyklus == "2J":
        zyklus = 365 * 2
    elif zyklus == "4J":
        zyklus = 365 * 4
    return zyklus

In [ ]:
merged_df["Zyklus Wartung"] = merged_df["Zyklus Wartung"].apply(zyklus_in_int)

In [ ]:
print(np.round(merged_df.max(), 2), "\n", np.round(merged_df.min(), 2))


In [ ]:
merged_df.to_csv('../data/merged_data.csv', index=False)

In [ ]:
# Separate the 'equipment_id' column
equipment_id = merged_df['Equipment ID']
features = merged_df.drop(columns=['Equipment ID'])

# Scale only the feature columns
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
features_scaled = pd.DataFrame(features_scaled, columns=features.columns)

In [ ]:
corr = features_scaled.corr()
print(corr)

In [ ]:
pca = PCA(n_components=3)
features_scaled_pca = pca.fit_transform(features_scaled)
features_scaled_pca = pd.DataFrame(features_scaled_pca, columns=["PCA", "PCA_2", "PCA_3"])

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
X_train = pd.concat([merged_df['Equipment ID'], features_scaled_pca], axis=1)

In [ ]:
dbscan = DBSCAN(eps=0.25, min_samples=100)
labels = dbscan.fit_predict(X_train[['PCA', 'PCA_2', 'PCA_3']])

In [ ]:
num_clusters = len(set(labels) - {-1})

print(f"Number of clusters: {num_clusters}")

In [ ]:
X_train.loc[:, 'cluster'] = labels

In [ ]:
fig = go.Figure()  # Create a 3D scatter plot using Plotly Graph Objects

fig.add_trace(go.Scatter3d(
    x=X_train["PCA"],
    y=X_train["PCA_2"],
    z=X_train["PCA_3"],
    mode='markers',
    marker=dict(
        size=8,
        color=X_train["cluster"],
        colorscale='Viridis',  # Color scale
        opacity=0.8,
        colorbar=dict(title='Cluster')
    ),
    hovertemplate='<b>PC:</b> %{z}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
))
# Update layout for better visualization
fig.update_layout(
    title='3D Scatter Plot of Equipment Clusters in Germany',
    scene=dict(
        xaxis=dict(title='Longitude'),
        yaxis=dict(title='Latitude'),
        zaxis=dict(title='PC'),
        aspectmode='cube'  # Ensure aspect ratio is equal
    )
)

fig.show()

In [ ]:
# Calculate centroid of each cluster
cluster_centers = X_train.groupby('cluster')[['PCA', 'PCA_2', 'PCA_3']].mean().reset_index()

reduce_maintenance = []
increase_maintenance = []

for centroid in cluster_centers.itertuples():
    print(centroid)
    if centroid.cluster == -1:
        continue
    if centroid.PCA >= 1:
        increase_maintenance.append(centroid)
    elif centroid.PCA < 5:
        reduce_maintenance.append(centroid)

reduce_maintenance = pd.DataFrame(reduce_maintenance)
increase_maintenance = pd.DataFrame(increase_maintenance)

In [ ]:
# Create a 3D scatter plot using Plotly Graph Objects
fig = go.Figure()

if len(reduce_maintenance) > 0:    
    reduce_maintenance_equipment = X_train[X_train['cluster'].isin(reduce_maintenance['cluster'])]
    
    fig.add_trace(go.Scatter3d(
        x=reduce_maintenance_equipment['lon'],
        y=reduce_maintenance_equipment['lat'],
        z=reduce_maintenance_equipment['PCA'],
        mode='markers',
        marker=dict(
            size=8,
            color='green',
            colorscale='Viridis',              # Color scale
            opacity=0.8,
            colorbar=dict(title='Cluster')
        ),
        hovertemplate='<b>PC:</b> %{z}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
    ))

if len(increase_maintenance) > 0:    
    increase_maintenance_equipment = X_train[X_train['cluster'].isin(increase_maintenance['cluster'])]
    
    fig.add_trace(go.Scatter3d(
        x=increase_maintenance_equipment['lon'],
        y=increase_maintenance_equipment['lat'],
        z=increase_maintenance_equipment['incident_count'],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            colorscale='Viridis',              # Color scale
            opacity=0.8,
            colorbar=dict(title='Cluster')
        ),
        hovertemplate='<b>PC:</b> %{z}<br><b>Lat:</b> %{y}<br><b>Lon:</b> %{x}'
    ))


# Update layout for better visualization
fig.update_layout(
    title='3D Scatter Plot of Equipment Clusters in Germany',
    scene=dict(
        xaxis=dict(title='Longitude'),
        yaxis=dict(title='Latitude'),
        zaxis=dict(title='PC'),
        aspectmode='cube'  # Ensure aspect ratio is equal
    )
)

fig.show()

# Save the plot as an HTML file
fig.write_html('../plots/result_PCA.html')

In [ ]:
# Assign colors

if len(increase_maintenance) > 0:
    increase_maintenance_equipment.loc[:, 'color'] = 'red'
else: 
    combined = reduce_maintenance_equipment
if len(reduce_maintenance) > 0:
    reduce_maintenance_equipment.loc[:, 'color'] = 'green'
else:
    combined = increase_maintenance_equipment
if len(reduce_maintenance) > 0 and len(increase_maintenance) > 0:
    combined = pd.concat([reduce_maintenance_equipment, increase_maintenance_equipment])

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Add geographical features
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.LAND, facecolor='white')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE, zorder=5)

# Plot data points
for color, group in combined.groupby('color'):
    print(f"Plotting color: {color} with {len(group)} points")  # Debugging statement
    ax.scatter(group['lon'], group['lat'], color=color, s=100, alpha=0.7, transform=ccrs.PlateCarree())

# Set plot title and extent
ax.set_title('Equipment Clusters in Germany')
ax.set_extent([5, 15, 47, 55], crs=ccrs.PlateCarree())

# Add legend
ax.legend(['Reduce Maintenance', 'Increase Maintenance'], loc='upper left')

# Save the plot as an image
plt.savefig('../plots/cluster_plot_incidents.svg', bbox_inches='tight')

# Show the plot
plt.show()